## Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [32]:
df = pd.read_csv(r'C:\Python\15. 30_04\ml-latest-small\ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [70]:
result = df.groupby("userId", as_index=False).aggregate({'timestamp': ['min', 'max', 'count']})
result.columns = [col[1] for col in result.columns]
result['duration'] = result['max'] - result['min'] 
avg_duration = result.loc[result['count']>100]['duration'].mean()
avg_duration

40080507.4496124

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [45]:

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [55]:
data_base1 = client_base.merge(rzd, how ='left').merge(air, how ='left').merge(auto, how ='left')
data_base1

,client_id,address,rzd_revenue,air_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,NaN,57483.0
3,114,Мира 14,5774.0,NaN,83.0
4,115,ЗЖБИиДК 1,981.0,81.0,912.0
5,116,Строителей 18,NaN,4.0,4834.0
6,117,Панфиловская 33,NaN,13.0,98.0
7,118,Мастеркова 4,NaN,173.0,NaN


In [60]:
data_base2 = data_base1
del data_base2['address']
data_base2

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0
5,116,NaN,4.0,4834.0
6,117,NaN,13.0,98.0
7,118,NaN,173.0,NaN


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

В таблицу визитов добавить координаты каждого визита по дате (первый и последний). Аналогично по таблице покупок.
Можно присвоить признак мобильности покупателю, если первые и последние координаты максимально удалены (отобрав юзеров, которые предпочитают покупки в дороге). 